# PDF Text + Graphical Data Q&A

This notebook documents the approach and demonstrates results for:

- Extracting **text** and **graphical data** (tables, basic bar charts) from a PDF.
- Converting those into structured, queryable data.
- Letting users **ask questions** against both text and derived data.

**Instructions:** Place your PDF under `../data/` and set its filename below.

In [19]:
# --- FIX WORKING DIRECTORY WHEN RUNNING AS ADMIN ---
import os
from pathlib import Path

# Always find the real project root dynamically
for potential_root in [
    Path.cwd(),
    Path(__file__).parent if "__file__" in globals() else Path.cwd(),
]:
    while not (potential_root / "data").exists() and potential_root != potential_root.parent:
        potential_root = potential_root.parent
    if (potential_root / "data").exists():
        break

os.chdir(potential_root)
print(f"✅ Working directory set to project root: {Path.cwd()}")


✅ Working directory set to project root: C:\Users\Al Justin Geronimo\rag-challenge


In [20]:
from dotenv import load_dotenv
import pandas as pd

from src.extract_text import extract_text
from src.extract_tables import extract_tables
from src.extract_charts import extract_charts_as_data
from src.qa import answer_table_query

DATA_DIR = Path("data")
OUTPUTS = Path("outputs")
OUTPUTS.mkdir(exist_ok=True, parents=True)

PDF_FILE = DATA_DIR / "test_info_extract.pdf"
assert PDF_FILE.exists(), f"❌ PDF not found at {PDF_FILE.resolve()}"

print(f"✅ PDF located: {PDF_FILE}")


✅ PDF located: data\test_info_extract.pdf


## 1) Extraction
We use:
- `pdfplumber` for text and page rasterization.
- `camelot` (stream/lattice) → `tabula-py` → `pdfplumber` fallback for tables.
- Simple computer vision + OCR to attempt digitizing **bar charts**.

> Note: Chart digitization is best-effort. Tables are the primary path to precise structured data.

In [21]:
from src.extract_text import extract_text
from src.extract_tables import extract_tables
from src.extract_charts import extract_charts_as_data

pages = extract_text(str(PDF_FILE))
tables = extract_tables(str(PDF_FILE), flavor="stream")
chart_dfs, chart_images = extract_charts_as_data(str(PDF_FILE), str(OUTPUTS/"images"))

len(pages), len(tables), len(chart_dfs), len(chart_images)

(2, 5, 2, 2)

### Quick Preview

In [22]:
pages[0].text[:1000] if pages else "(no text)"

'Home Energy Report:\nelectricity\nMarch report\nAccount number: 954137\nService address: 1627 Tulip Lane\nFind your personalized\nanalysis of your electrical\nenergy use. Scan this code\nor log in to your account at\nfranklinenergy.com.\nDear JILL DOE, here is your usage analysis for March.\nYour electric use: 18% more than similar nearby homes\nYou\n125 kWh\nAbove\nSimilar nearby homes\n103 kWh\ntypical use\nEfficient nearby homes\n49 kWh\nNearby homes are defined as... Monthly savings tip: Do full\nlaundry loads.\nOther homes with electricity\nWaiting until you\nhave a full load to\nHomes within 9 km\nrun your laundry\ncan save up to 6%\nHomes within +/- 300 sq. ft. of your energy use.\nWatch this space\nNearby homes are based on fuel, distance and size. Square footage\nis collected from public information sources. Efficient nearby homes\nfor new ways to save energy each month.\nare the top 15 per cent efficient of similar-sized homes nearby.\nTurn over for more savings ideas.'

In [23]:
tables[0].head() if tables else "(no tables)"

,
0,


In [24]:
chart_dfs[0].head() if chart_dfs else "(no chart data detected)"

,label,value_scaled
0,Your,0.000000
1,electric,0.000000
2,use:,0.000000
3,Above,0.666667
4,18%,0.000000


## 2) Build Searchable Index for Text

In [25]:
from src.build_index import build_text_index, TextChunk, tables_summary
import os

model_name = os.getenv("EMBEDDING_MODEL", "sentence-transformers/all-MiniLM-L6-v2")
chunks = [TextChunk(page_num=p.page_num, text=p.text) for p in pages if p.text.strip()]
text_index = build_text_index(chunks, model_name=model_name)

print("Text chunks:", len(chunks))
tables_summary(tables)

Text chunks: 2


['Table 1 with 1 rows, columns: ',
 'Table 2 with 1 rows, columns: Find your personalized\nanalysis of your electrical\nenergy use. Scan this code\nor log in to your account at\nfranklinenergy.com., , None, , None',
 'Table 3 with 1 rows, columns: , ',
 'Table 4 with 2 rows, columns: , , None',
 'Table 5 with 2 rows, columns: , ']

## 3) Ask Questions
We support:
- **Semantic text search** over the narrative text.
- **Table/derived data queries** for simple aggregations and previews.

In [26]:
from src.qa import answer_text_query, answer_table_query

q1 = "What does the document say about revenue growth?"
ans1 = answer_text_query(text_index, q1, top_k=int(os.getenv("TOP_K", 5)))
ans1

{'matches': [{'page': 1,
   'score': 0.0997,
   'text': 'Home Energy Report:\nelectricity\nMarch report\nAccount number: 954137\nService address: 1627 Tulip Lane\nFind your personalized\nanalysis of your electrical\nenergy use. Scan this code\nor log in to your account at\nfranklinenergy.com.\nDear JILL DOE, here is your usage analysis for March.\nYour electric use: 18% more than similar nearby homes\nYou\n125 kWh\nAbove\nSimilar nearby homes\n103 kWh\ntypical use\nEfficient nearby homes\n49 kWh\nNearby homes are defined as... Monthly savings tip: Do full\nlaundry loads.\nOther homes with electricity\nWaiting until you\nhave a full load to\nHomes within 9 km\nrun your laundry\ncan save up to 6%\nHomes within +/- 300 sq. ft. of your energy use.\nWatch this space\nNearby homes are based on fuel, distance and size. Square footage\nis collected from public information sources. Efficient nearby homes\nfor new ways to save energy each month.\nare the top 15 per cent efficient of similar-size

In [27]:
# After extraction
pdf_path = "data/test_info_extract.pdf"
tables = extract_tables(pdf_path)
# extract_charts_as_data returns (list_of_dfs, list_of_chart_paths)
chart_dfs, chart_images = extract_charts_as_data(pdf_path, "outputs/charts")

# Combine all DataFrames only
valid_tables = [
    t for t in (tables + chart_dfs)
    if isinstance(t, pd.DataFrame)
    and t.shape[1] > 1
    and t.dropna(how="all").shape[0] > 0
]

print(f"Filtered valid tables: {len(valid_tables)} found")

# Query
from src.qa import answer_table_query

q2 = "sum of Revenue where Year == 2023"
ans2 = answer_table_query(valid_tables, q2)
ans2


Filtered valid tables: 6 found


{'table_index': 0,
 'preview': [{'Find your personalized\nanalysis of your electrical\nenergy use. Scan this code\nor log in to your account at\nfranklinenergy.com.': nan}]}

## 4) Notes on Design Choices & Challenges

1. **Multi-backend table extraction**: PDFs vary widely. We layer Camelot → Tabula → pdfplumber to maximize table recovery.
2. **Chart digitization (best-effort)**: Simple bar charts can often be parsed by detecting vertical rectangles and scaling bar heights. Arbitary plots (line charts, stacked bars, complex legends) are left as future work.
3. **Embeddings**: We use local Sentence-Transformers to avoid external APIs. Cosine similarity over normalized vectors provides robust semantic search across pages.
4. **NL-to-table**: A tiny pattern parser recognizes queries like `sum of <col> where <col2> == X`. When unrecognized, we surface a **preview** of likely columns as a helpful fallback.
5. **Provenance**: We retain page numbers with text chunks, and we keep DataFrames for tables so you can trace results back to source pages.

**Limitations**
- Tabula requires Java; if missing, those steps are skipped.
- Ghostscript improves Camelot lattice detection.
- Chart digitization is intentionally conservative: it will skip when uncertain rather than hallucinate numbers.
